In [16]:
import requests
import json
import itertools


In [17]:
# Initial requests from API url (23.08.19)
config = readJson("config.json")
client_id=config["client_id"]
elev_key = config["elevation_key"]
test_area = config["test_area"]

#Mapillary
def requestImageById(image_key):
    request = requests.get("https://a.mapillary.com/v3/images/"+image_key+"?client_id="+client_id)
    return request.json()

def requestSequencesByBbox(ul, lr):
    # i prefer [lat, lng], so have to keep that in mind
    # request of bbox is box=minx,miny,maxx,maxy=ul_lng, lr_lat, lr_lng, ul_lat
    request = requests.get(f"https://a.mapillary.com/v3/sequences?bbox={ul[1]},{lr[0]},{lr[1]},{ul[0]}&per_page=10000&client_id={client_id}")
    return request.json()

def requestSequenceById(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/sequences/"+sequence_key+"?client_id="+client_id)
    return request.json()

def requestImagesBySequence(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/images/?sequence_keys="+sequence_key+"&per_page=10000&client_id="+client_id)
    return request.json()

In [44]:
# Mapillary API allows max 1000 sequences per area, so if the number of sequences>1000, i split in 4 and request for new areas
def get_all_sequences_in_area(bbox):
    ul, lr = bbox
    areas = [[ul, lr]]
    done = []
    max_retrieved = 1000
    while(True):
        new_areas = []
        for (ul_i, lr_i) in areas:   
            request = requestSequencesByBbox(ul_i, lr_i)
            if(len(request["features"])<max_retrieved):
                done.append(request["features"])
            else:
                split_lat = ul_i[0]+(lr_i[0]-ul_i[0])/2
                split_lng = lr_i[1]+(ul_i[1]-lr_i[1])/2
                split = [split_lat, split_lng]
                new_areas.append([ul_i,split])
                new_areas.append([[split[0], ul_i[1]],[lr_i[0], split[1]]])
                new_areas.append([[ul_i[0], split[1]],[split[0], lr_i[1]]])
                new_areas.append([split, lr_i])
        if(len(new_areas)==0):
            break
        else:
            areas = new_areas
    
    return {
        "type":"FeatureCollection",
        "features":list(itertools.chain.from_iterable(done))
    }

def write_data(data):
    to_file = "mapillary_data.json"
    features = []
    for i, seq in enumerate(data["features"]):
        _id =seq["properties"]["key"]
        display(images = requestImagesBySequence(_id)["features"])
        features.append({
            "id":_id,
            "images":images
        })
        
    with open(to_file, "w+") as f:
        json.dump({
            "type":"FeatureCollection",
            "features":features
        }, f)

In [29]:
data = get_all_sequences_in_area(test_area)

In [43]:
write_data(data)